In [83]:
import cv2
import numpy as np

def remove_small_components(mask, min_size):
    # 連通區域標記
    _, labeled_mask, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    
    # 過濾掉小於指定大小的區塊
    for i in range(1, labeled_mask.max() + 1):
        if stats[i, cv2.CC_STAT_AREA] < min_size:
            mask[labeled_mask == i] = 0

    return mask

In [84]:
def segment_sky(image_path):
    # 讀取圖片
    img = cv2.imread(image_path)
    # 轉換到HSV色彩空間
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # 圖片上半部
    sky_region = hsv_img[:img.shape[0]//2, :, :]

    # 天空區域平均HSV
    avg_hsv = np.mean(sky_region, axis=(0, 1))

    # HSV範圍
    h_range = 25
    s_range = 110
    v_range = 70

    lower_bound = np.array([avg_hsv[0] - h_range, 0 if avg_hsv[1] - s_range == 0 else avg_hsv[1] - s_range, 0 if avg_hsv[2] - v_range == 0 else avg_hsv[2] - v_range])
    upper_bound = np.array([avg_hsv[0] + h_range, 255 if avg_hsv[1] + s_range == 255 else avg_hsv[1] + s_range, 255 if avg_hsv[2] + v_range == 255 else avg_hsv[2] + v_range])


    mask = cv2.inRange(hsv_img, lower_bound, upper_bound)

    # 後處理
    
    mask[:mask.shape[0]//6] = 1

    # kernel = np.ones((8, 8), np.uint8)
    # final_mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    # final_mask = cv2.blur(mask, (9, 9))   # 指定區域單位為 (5, 5)
    kernel = np.ones((9, 9), np.uint8)
    final_mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    kernel = np.ones((9, 9), np.uint8)
    final_mask = cv2.morphologyEx(final_mask, cv2.MORPH_OPEN, kernel)
    kernel = np.ones((9, 9), np.uint8)

    final_mask = cv2.erode(final_mask, kernel, iterations=2)  

    final_mask = remove_small_components(final_mask, (img.shape[0]*img.shape[1])/10)
    
    final_mask1 = cv2.dilate(final_mask, kernel, iterations=6)  
    final_mask = cv2.erode(final_mask1, kernel, iterations=9)  
    final_mask2 = cv2.erode(final_mask1, kernel, iterations=6)  
    

    # 分割出的天空區域
    img[final_mask == 0] = [0, 0, 0]

    # # 分割出的天空以外區域
    # img2[final_mask2 == 0] = [255, 0, 0]

    # 儲存結果
    # cv2.imshow("2",img)
    # cv2.waitKey(0)
    return img,final_mask2


In [85]:
def detect_stars_gradient(img, output_path, min_star_size, threshold):
    # 讀取圖片
    # img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # # 調整亮度
    # img = np.clip(img * 1.1, 0, 255).astype(np.uint8)

    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 圖像平滑處理
    img_smooth = cv2.GaussianBlur(img, (3, 3), 0)

    edges = cv2.Canny(img_smooth, 50, 150)

    _, thresh = cv2.threshold(edges, threshold, 255, cv2.THRESH_BINARY)

    # 將二值化圖像轉換為 uint8 格式
    thresh = thresh.astype(np.uint8)

    # 星星檢測
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 存儲每個星星的中心點座標
    star_centers = []


    result_img = np.zeros_like(img)
    for contour in contours:
        cv2.drawContours(result_img, [contour], -1, 255, thickness=cv2.FILLED)

    # 濾除雜訊
    kernel = np.ones((3, 3), np.uint8)
    result_img = cv2.morphologyEx(result_img, cv2.MORPH_CLOSE, kernel)
    result_img = cv2.morphologyEx(result_img, cv2.MORPH_OPEN, kernel)

    max_brightness = 0
    largest_star_area = 0
    brightest_largest_star_center = None

    
    # 結果尋找一次星星
    contours, _ = cv2.findContours(result_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        area = cv2.contourArea(contour)
        if area >= 0:
            # 計算星星中心
            M = cv2.moments(contour)
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            # 計算半徑
            star_coordinates = contour.squeeze()
            distances = np.linalg.norm(star_coordinates - np.array([cx, cy]), axis=1)
            average_radius = np.mean(distances)
            # 儲存中心
            star_centers.append((cx, cy, average_radius))

    # 將所有星星的中心點座標轉換為 NumPy 數組
    star_centers_np = np.array(star_centers)

    # 找到最亮、最大的星星
    for contour in contours:
        area = cv2.contourArea(contour)
        if area >= 0:
            # 計算星星區域的亮度
            mask = np.zeros_like(img)
            cv2.drawContours(mask, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)
            brightness = np.sum(cv2.bitwise_and(img, mask))

            # 找到最亮、最大的星星
            if brightness > max_brightness or (brightness == max_brightness and area > largest_star_area):
                max_brightness = brightness
                largest_star_area = area
                # 計算星星中心
                M = cv2.moments(contour)
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                brightest_largest_star_center = (cx, cy)

    # 儲存結果
    cv2.imwrite(output_path, result_img)
    return brightest_largest_star_center, star_centers_np


In [86]:
# def rotate_stars(org_img,output_path, center, all_stars, angle_degrees):
#     img = cv2.imread(org_img)
    
#     for point_on_circle in all_stars:
#        # 獲取座標和大小
#         coord = point_on_circle[:2]
#         size = point_on_circle[2]
#         color = img[int(coord[1]), int(coord[0]), :].tolist()

#         # 計算半徑
#         radius = np.linalg.norm(coord - center)

#         # 計算半圓的極坐標
#         start_angle = np.arctan2(coord[1] - center[1], coord[0] - center[0])
#         end_angle = start_angle + np.radians(angle_degrees)
#         angles = np.linspace(start_angle, end_angle, int((img.shape[0])*img.shape[1]/2000))
#         polar_coordinates = np.column_stack((radius * np.cos(angles), radius * np.sin(angles)))

#         # # 將極坐標轉換為直角坐標
#         # circle_coordinates = polar_coordinates + center
#         # 在新的直角坐標上繪製星星的半圓
#         scaled_size = size  # 初始大小
#         for coord1 in polar_coordinates:
#             color = img[int(coord[1]), int(coord[0]), :].tolist()
#             cv2.circle(img, (int(coord1[0] + center[0]), int(coord1[1] + center[1])), int(scaled_size), color)
#             scaled_size *= 0.95  # 縮小 0.95 倍

#     # 顯示結果
#     cv2.imwrite(output_path, img)



In [122]:
from PIL import Image, ImageDraw
import numpy as np


def rotate_stars_pil(org_img, output_path,final_mask2, center, all_stars, angle_degrees):
    img_org = cv2.imread(org_img)

    img = Image.open(org_img)
    draw = ImageDraw.Draw(img)
    width, height = img.size

    for point_on_circle in all_stars:
        # 獲取座標和大小
        coord = point_on_circle[:2]
        size = point_on_circle[2]

        # 計算半徑
        radius = np.linalg.norm(np.array(coord) - np.array(center))

        # 計算半圓的極坐標
        start_angle = np.arctan2(coord[1] - center[1], coord[0] - center[0])
        end_angle = start_angle + np.radians(angle_degrees)
        angles = np.linspace(start_angle, end_angle, int(width * height / 2000))

        # 在新的直角坐標上繪製星星的半圓
        scaled_size = size  # 初始大小
        for angle in angles:
            x = int(radius * np.cos(angle) + center[0])
            y = int(radius * np.sin(angle) + center[1])
            draw.point((x, y), fill=tuple(img.getpixel((int(coord[0]), int(coord[1])))))
            scaled_size *= 0.95  # 縮小 0.95 倍
    
    # 貼上前景
    img_array = np.array(img)
    img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
    img_array[final_mask2 == 0] = img_org[final_mask2 == 0]
    # 保存結果
    cv2.imwrite(output_path, img_array)





In [88]:
from PIL import Image, ImageDraw
import numpy as np
from tqdm import tqdm

def rotate_stars_GIF(org_img, output_path,final_mask2, center, all_stars, angle_degrees):
    img = Image.open(org_img)
    img_org = cv2.imread(org_img)
    width,hight = img.size
    num_frames = 30  # 幀數

    frames = []

    for i in tqdm(range(num_frames), desc="Generating Frames"):
        frame_img = img.copy()
        draw = ImageDraw.Draw(frame_img)

        for point_on_circle in all_stars:
            coord = point_on_circle[:2]
            size = point_on_circle[2]
            # color = tuple(img1[int(coord[1]), int(coord[0]), :])
            color = img.getpixel((coord[0], coord[1]))
            radius = np.linalg.norm(np.array(coord) - np.array(center))
            start_angle = np.arctan2(coord[1] - center[1], coord[0] - center[0])
            end_angle = start_angle + np.radians(angle_degrees * (i + 1) / num_frames)
            angles = np.linspace(start_angle, end_angle, int(width * hight / 2000))
            polar_coordinates = np.column_stack((radius * np.cos(angles), radius * np.sin(angles)))

            # 在新的直角坐標上繪製星星的半圓
            scaled_size = size  # 初始大小
            for angle in angles:
                x = int(radius * np.cos(angle) + center[0])
                y = int(radius * np.sin(angle) + center[1])
                draw.point((x, y), fill=tuple(img.getpixel((int(coord[0]), int(coord[1])))))
                scaled_size *= 0.95  # 縮小 0.95 倍
        img_array = np.array(frame_img)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        img_array[final_mask2 == 0] = img_org[final_mask2 == 0]

        # 在這裡確認轉換格式
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

        frames.append(Image.fromarray(img_array))

    frames[0].save(output_path, save_all=True, append_images=frames[1:], duration=100, loop=0, disposal=2)


In [127]:
import time

# 替換成你的圖片路徑和輸出路徑
dark_image_path = "Final project Dataset/dark foreground/foreground_2.jpg"
light_image_path = "Final project Dataset/light foreground/light_foreground_1.jpg"
dark_output_image_path= "output/hsv/dark_multi_threshold2.png"
light_output_image_path= "output/hsv/light_multi_threshold1.png"

min_star_size = 1
threshold_value = 200
rotation_angle = 30

for i in range (1,7):
        # if i ==2:
        #         continue
        # else:
                start = time.time()
                img, img_bgra = segment_sky("Final project Dataset/aurora/aurora_" + str(i) + ".jpg")
                end = time.time()
                print("done get sky：%f s" % (end - start))
                start = time.time()
                center,star_coords = detect_stars_gradient(img, "output/hsv/star/light_" + str(i) + ".png",min_star_size,threshold_value)
                end = time.time()
                print("done get star：%f s" % (end - start))
                start = time.time()
                rotate_stars_pil("Final project Dataset/aurora/aurora_" + str(i) + ".jpg", "output/final/aurora_" + str(i) + ".png",img_bgra, center, star_coords, rotation_angle)
                end = time.time()
                print("done get startrail：%f s" % (end - start))




done get sky：3.806792 s
done get star：12.555727 s
done get startrail：32.578339 s
done get sky：2.942133 s
done get star：23.506575 s
done get startrail：58.812876 s
done get sky：1.211306 s
done get star：3.276237 s
done get startrail：8.857333 s
done get sky：0.047845 s
done get star：0.102753 s
done get startrail：0.498603 s
done get sky：0.479745 s
done get star：0.976387 s
done get startrail：2.792834 s
done get sky：0.359463 s
done get star：2.823410 s
done get startrail：7.581718 s


In [129]:
import time

min_star_size = 1
threshold_value = 200
rotation_angle = 30

for i in range (1,7):
        # if i ==2:
        #         continue
        # else:
                start = time.time()
                img, img_bgra = segment_sky("Final project Dataset/aurora/aurora_" + str(i) + ".jpg")
                end = time.time()
                print("done get sky：%f s" % (end - start))
                start = time.time()
                center,star_coords = detect_stars_gradient(img, "output/hsv/star/light_" + str(i) + ".png",min_star_size,threshold_value)
                end = time.time()
                print("done get star：%f s" % (end - start))
                start = time.time()
                rotate_stars_GIF("Final project Dataset/aurora/aurora_" + str(i) + ".jpg", "output/final/aurora_" + str(i) + ".gif", img_bgra,center, star_coords, rotation_angle)
                end = time.time()
                print("done get startrail：%f s" % (end - start))




done get sky：4.065414 s
done get star：13.388079 s


Generating Frames: 100%|██████████| 30/30 [16:06<00:00, 32.21s/it]


done get startrail：1118.069740 s
done get sky：3.072818 s
done get star：23.839701 s


Generating Frames: 100%|██████████| 30/30 [29:31<00:00, 59.06s/it]


done get startrail：1846.607022 s
done get sky：1.124010 s
done get star：3.004885 s


...

done get startrail：19.431904 s
done get sky：0.503386 s
done get star：0.985626 s


Generating Frames: 100%|██████████| 30/30 [01:18<00:00,  2.60s/it]


done get startrail：108.923996 s
done get sky：0.384968 s
done get star：2.713453 s


Generating Frames: 100%|██████████| 30/30 [03:37<00:00,  7.26s/it]


done get startrail：259.234975 s
